In [ ]:
!pip install peft transformers datasets accelerate bitsandbytes evaluate



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from datasets import Dataset
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, XLNetTokenizer, XLNetForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
import evaluate


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/balanced_subset.csv')


# Sample 10% of the data
sampled_data = data.sample(frac=0.3, random_state=42)
sampled_data.rating.value_counts()

,count
rating,
Positive,195015
Negative,194611
Neutral,88119


In [ ]:
sampled_data

,rating,cleaned_text
913868,Positive,situate right main street manayunk luckys last...
695928,Neutral,go th anniversary groupon present service know...
211072,Negative,ive sit hr half wait food come buddy gift card...
324554,Neutral,mean check place go one magazine street love o...
136857,Negative,shop time get cute outfit happy one bra bottom...
...,...,...
688355,Negative,terrible service late pay hour work less hour ...
1088820,Positive,ive come year develop rapport barber get great...
1407816,Positive,food atmosphere incredible get breakfast tacos...
968816,Positive,authentic great mexican food probably best tre...


In [ ]:
# Set the device: TPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {'GPU' if torch.cuda.is_available() else 'CPU'}")
model_name = "bert-base-uncased"
# Load a pre-trained BERT tokenizer

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)


Using GPU


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Encode the labels
# label_encoder = LabelEncoder()
# sampled_data['rating'] = label_encoder.fit_transform(sampled_data['rating'])

# Convert ratings to numerical labels if they are categorical
sampled_data['rating'] = sampled_data['rating'].map({"Positive": 2, "Neutral": 1, "Negative": 0})

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sampled_data['cleaned_text'].tolist(), sampled_data['rating'].tolist(), test_size=0.2, random_state=42
)

# Check the splits
print(f"Training dataset: {len(train_texts)}")
print(f"Test dataset: {len(val_texts)}")

Training dataset: 382196
Test dataset: 95549


In [ ]:
# Tokenize text using multiple processes
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

In [ ]:
# Tokenize text using multiple processes
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
# Combine tokenized encodings and labels into Hugging Face Dataset
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels,
})

val_dataset = Dataset.from_dict({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": val_labels,
})

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Task: Sequence Classification
    inference_mode=False,       # Enable training mode
    r=8,                        # LoRA rank
    lora_alpha=16,              # Scaling factor
    lora_dropout=0.1,           # Dropout for LoRA
)

# Apply PEFT (LoRA)
model = get_peft_model(model, lora_config).to(device)
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [ ]:
# Load accuracy metric
accuracy = evaluate.load("accuracy")

In [ ]:
# Load accuracy metric
accuracy = evaluate.load("accuracy")
# Define metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Enable mixed precision for CUDA
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-17-2785c5a88f2d>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels,
})

val_dataset = Dataset.from_dict({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": val_labels,
})

# Continue training as usual
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-18-f46dfb52ed4b>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.445500,0.456144,0.813541
2,0.631800,0.436842,0.819527
3,0.445300,0.433893,0.821107


TrainOutput(global_step=71664, training_loss=0.46852009452847826, metrics={'train_runtime': 7078.8373, 'train_samples_per_second': 161.974, 'train_steps_per_second': 10.124, 'total_flos': 7.568239683485491e+16, 'train_loss': 0.46852009452847826, 'epoch': 3.0})

In [ ]:
# Evaluate the model on the validation set after training
eval_results = trainer.evaluate()

# Print the evaluation report
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")

Evaluation Results:
eval_loss: 0.43389272689819336
eval_accuracy: 0.8211074945839307
eval_runtime: 223.8887
eval_samples_per_second: 426.77
eval_steps_per_second: 26.674
epoch: 3.0


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Machine Learning/fine_tuned_lora")
tokenizer.save_pretrained("/content/drive/MyDrive/Machine Learning/fine_tuned_lora")

('/content/drive/MyDrive/Machine Learning/fine_tuned_lora/tokenizer_config.json',
 '/content/drive/MyDrive/Machine Learning/fine_tuned_lora/special_tokens_map.json',
 '/content/drive/MyDrive/Machine Learning/fine_tuned_lora/vocab.txt',
 '/content/drive/MyDrive/Machine Learning/fine_tuned_lora/added_tokens.json')